## Dataframe Basics

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-06 23:04:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.74MB/s    in 0.2s    

2022-03-06 23:04:23 (5.74 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").config("spark.jars","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, quote="")

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [ ]:
df.count()

4880466

In [ ]:
from pyspark.sql.functions import col
df_review_id_table = df.select(['review_id','customer_id','product_id','product_parent','review_date']).distinct()
df_review_id_table = (df_review_id_table
                      .withColumn("customer_id", col("customer_id").cast("int"))
                      .withColumn("product_parent", col("product_parent").cast("int"))
                      .withColumn("review_date", col("review_date").cast("date")))
df_review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
df_review_id_table.count()


4880466

In [ ]:
df_review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1003WQ2N568UJ|   30984122|B000HCJX2C|     561470701| 2015-07-16|
|R1004EZA3OASMT|   15509407|B0009YYQ4S|     455866213| 2011-12-14|
|R100EXNK731LTK|   37164434|B0038U5WO8|     233057226| 2015-05-25|
|R100PQSJMOLWP0|   26963189|B001ICPAZE|     709838409| 2011-12-28|
|R10168116JX8X2|   30974720|B0018CG2EC|     500296269| 2013-11-12|
|R10173AGXELGVQ|   33160607|B002DX8QNU|     602831505| 2015-08-28|
|R101EV570FB01A|   12321022|B00PUVABGC|     298139833| 2015-03-26|
|R101JMABBJBX6U|   51876859|B000W5SLB8|     956903177| 2014-05-03|
|R102BUG3QZXMY8|   11142660|B001MUPGQ0|     266396365| 2014-04-16|
|R102OI838NN3Z7|   11056168|B000YDUAMQ|     180607962| 2014-01-18|
|R102VGUEHU4MVS|   26684756|B0018CE8LQ|     855508227| 2014-07-16|
|R103E8CGPOZD6B|    8996047|B0053AS5VS|     840020528| 2014-01

In [ ]:
df_products = df.select(['product_id','product_title']).distinct()
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
df_products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004NWK4W0|Wildkin Pink Leop...|
|B00IE70YFC|Wilton 415-2178 3...|
|B00004W499|George Foreman GG...|
|B000Q945FG|BIA Cordon Bleu B...|
|B007WTOIPA|Clorox Sales Co B...|
|B006IC09P0|Utenlid Acrylic R...|
|B00CDSU2MI|2000 Diamond Tabl...|
|B000WJMTNA|OXO Good Grips St...|
|B00DYRI13O|Reed & Barton 148...|
|B005FPD67Y|     Ergo Chef Tongs|
|B00JLA6POU|TeChef - Art Pan ...|
|B00FN3P7X4|Hamilton Beach 1....|
|B000UVHUCU|KitchenAid Ultra ...|
|B00IZCWYR2|PackIt Freezable ...|
|B0105V7VS2|Top Rated Lemon S...|
|B002P667TU|Browne (22126P) 6...|
|B00KIYWDTO|Wolfgang Puck Pre...|
|B00004S9EJ|Cuisinart DLC-8SY...|
|B00D19SSE2|Ozeri Serafino Do...|
|B00JV8PTYY|Thermos 12 Ounce ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#df_customers = df_review_id_table.groupBy("customer_id").count().withColumnRenamed('count','customer_count')
#df_customers.show()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cvhifjxfcutl.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"user", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to table in RDS 

df_review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
df_products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
#df_customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored